In [1]:
import numpy as np
import individual_methods as estimate
from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression
import pandas as pd
import time

#read in data
data = pd.read_csv('GDP_data.csv')

#select the last 100 periods and then scale data by 1e-8
GDP = data.values[:,-100:].astype(float)*1e-8
units, periods = GDP.shape

#GDP growth rate
GDP = data.values[:,-101:].astype(float)
GDP = (GDP[:,1:]-GDP[:,:-1])/GDP[:,:-1]*1e2
units, periods = GDP.shape

#specify the number of test and train periods
test_periods = 10
train_periods = periods-test_periods

#record RMSE of predictions by 9 methods
RMSE_test = np.zeros((units,test_periods,9))

#ensemble weights
ensem1_weights = np.zeros((units,5))
ensem2_weights = np.zeros((units,5))
ensem3_weights = np.zeros((units,5))
ensem4_weights = np.zeros((units,5))
ensem4_intercepts = np.zeros(units,)

#to calculate correlation of predictions by different methods
Epsilon = np.zeros((units,test_periods,5))

#outcome matrix for training to obtain ensemble weights
Y = GDP[:,:-test_periods]

#matrix of RMSE for ensemble weighting
RMSE_weight = np.zeros((units,train_periods,5))

#for each unit, obtain weights from train periods first, then 
#apply individual and ensemble methods to test periods

#train for training periods
for current_unit in range(units):
    
    #shift current unit to the last row
    Y0 = np.roll(Y,-current_unit-1,0)
    
    #to save computation time, obtain regularization from cross-validation
    #for last period of current unit and use that for all train periods
    _, HR_alpha, HR_l1_ratio = estimate.HR(Y0,cv=True)
    _, SC_alpha, SC_l1_ratio = estimate.SC(Y0,cv=True)
    
    #record predictions of individual methods and actual outcome for 
    #regression ensemble weighting
    individual_predictions = np.zeros((train_periods,5))
    actual_outcomes = np.zeros(train_periods,)
    
    for train_period in range(train_periods):
        
        #shift current train period to the last column
        Y_shift = np.roll(Y0,-train_period-1,1)
        actual_outcomes[train_period] = Y_shift[-1,-1]
        
        #obtain RMSE for ensemble weighting
        
        #METHOD-1 MC
        individual_predictions[train_period,0] = estimate.MC(Y_shift)
        
        #METHOD-2 HR
        individual_predictions[train_period,1] = estimate.HR(Y_shift,a=HR_alpha,b=HR_l1_ratio)
        
        #METHOD-3 SC
        individual_predictions[train_period,2] = estimate.SC(Y_shift,a=SC_alpha,b=SC_l1_ratio)
        
        #METHOD-4 Abadie
        individual_predictions[train_period,3] = estimate.Abadie(Y_shift)
        
        #METHOD-5 DIDs
        individual_predictions[train_period,4] = estimate.DID(Y_shift)
        
        RMSE_weight[current_unit,train_period,:] = np.absolute(individual_predictions[train_period,:]-actual_outcomes[train_period])
        
    #obtain ensemble weights for regression ensemble methods
    ensem3_weights[current_unit,:] = estimate.restricted_regression(individual_predictions,actual_outcomes)
    
    reg = LinearRegression().fit(individual_predictions, actual_outcomes)
    
    ensem4_weights[current_unit,:] = reg.coef_
    ensem4_intercepts[current_unit] = reg.intercept_
 
#obtain inverse-RMSE ensemble weighting
RMSE_weight = np.squeeze(np.sum(np.square(RMSE_weight),1))
RMSE_weight = np.sqrt(RMSE_weight)

inv_RMSE = np.reciprocal(RMSE_weight)

ensem2_weights = (inv_RMSE.T/np.sum(inv_RMSE,1)).T

#benchmark of equal weighting
ensem1_weights[:,:] = 0.2;        
        
#predict for test periods
for test_period in range(test_periods):
    
    #select outcomes up to the current (to be predicted) period
    Y = GDP[:,:-test_periods+test_period+1]
    
    for current_unit in range(units):
    
        #shift current unit to the last row
        Y0 = np.roll(Y,-current_unit-1,0)
        
        #record predictions of individual methods
        individual_predictions = np.zeros(5,)
        
        #compare with actual outcome 
        actual_outcome = Y0[-1,-1]
        
        #METHOD-1 MC
        individual_predictions[0] = estimate.MC(Y0)
        
        #METHOD-2 HR
        individual_predictions[1],_,_ = estimate.HR(Y0,cv=True)
        
        #METHOD-3 SC
        individual_predictions[2],_,_ = estimate.SC(Y0,cv=True)
        
        #METHOD-4 Abadie
        individual_predictions[3] = estimate.Abadie(Y0)
        
        #METHOD-5 DIDs
        individual_predictions[4] = estimate.DID(Y0)
        
        Epsilon[current_unit,test_period,:] = individual_predictions
        
        RMSE_test[current_unit,test_period,0:5] = np.absolute(Epsilon[current_unit,test_period,:]-actual_outcome)
        
        #METHOD-6 ensemble with equal weighting
        ensem_prediction = np.dot(ensem1_weights[current_unit,:],individual_predictions)
        RMSE_test[current_unit,test_period,5] = np.absolute(ensem_prediction-actual_outcome)
        
        #METHOD-7 ensemble with inverse RMSE weighting
        ensem_prediction = np.dot(ensem2_weights[current_unit,:],individual_predictions)
        RMSE_test[current_unit,test_period,6] = np.absolute(ensem_prediction-actual_outcome)
        
        #METHOD-8 ensemble with restricted regression weighting
        ensem_prediction = np.dot(ensem3_weights[current_unit,:],individual_predictions)
        RMSE_test[current_unit,test_period,7] = np.absolute(ensem_prediction-actual_outcome)
        
        #METHOD-9 ensemble with unrestricted regression weighting
        ensem_prediction = np.dot(ensem4_weights[current_unit,:],individual_predictions)+ensem4_intercepts[current_unit]
        RMSE_test[current_unit,test_period,8] = np.absolute(ensem_prediction-actual_outcome)       

#calculate RMSE by averaging over test periods, i.e. axis 1        
RMSE_final = np.squeeze(np.sqrt(np.sum(np.square(RMSE_test),1)/test_periods))
RMSE_final = (RMSE_final.T/np.std(GDP,axis=1)).T
overall = np.sqrt(np.sum(np.square(RMSE_final),0)/units)
med = np.median(RMSE_final,0);

RMSE_output = np.concatenate((RMSE_final,np.reshape(overall,(1,-1)),np.reshape(med,(1,-1))),axis=0)

ensem4_coef = np.concatenate((ensem4_weights,np.reshape(ensem4_intercepts,(-1,1))),axis=1)

#calculate correlation and covariance of individual methods
cov_input = np.reshape(Epsilon.transpose(),(5,-1))

covariance_matrix = np.cov(cov_input)
correlation_matrix = np.corrcoef(cov_input)

index1 = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','mean','median']
columns1 = ['MC','HR','SC','Abadie','DID','ensem1','ensem2','ensem3','ensem4']
RMSE_print = pd.DataFrame(RMSE_output,index=index,columns=columns1)  

index2 = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
columns2 = ['MC','HR','SC','Abadie','DID']

corr_print = pd.DataFrame(correlation_matrix,index=columns2,columns=columns2)
ensem3_print = pd.DataFrame(ensem3_weights,index=index2,columns=columns2)

columns3 = ['MC','HR','SC','Abadie','DID','intercept']
ensem4_print = pd.DataFrame(ensem4_weights,index=index2,columns=columns3)
    


Using TensorFlow backend.
c:\users\zhaon\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\zhaon\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\zhaon\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[SoftImpute] Max Singular Value of X_init = 101.722712
[SoftImpute] Iter 1: observed MAE=0.160212 rank=44
[SoftImpute] Iter 2: observed MAE=0.160252 rank=44
[SoftImpute] Iter 3: observed MAE=0.160222 rank=44
[SoftImpute] Iter 4: observed MAE=0.160202 rank=44
[SoftImpute] Iter 5: observed MAE=0.160192 rank=44
[SoftImpute] Iter 6: observed MAE=0.160188 rank=44
[SoftImpute] Iter 7: observed MAE=0.160186 rank=44
[SoftImpute] Iter 8: observed MAE=0.160185 rank=44
[SoftImpute] Iter 9: observed MAE=0.160185 rank=44
[SoftImpute] Stopped after iteration 9 for lambda=2.034454
[SoftImpute] Max Singular Value of X_init = 101.731162
[SoftImpute] Iter 1: observed MAE=0.160208 rank=44
[SoftImpute] Iter 2: observed MAE=0.160171 rank=44
[SoftImpute] Iter 3: observed MAE=0.160129 rank=44
[SoftImpute] Iter 4: observed MAE=0.160105 rank=44
[SoftImpute] Iter 5: observed MAE=0.160093 rank=44
[SoftImpute] Iter 6: observed MAE=0.160087 rank=44
[SoftImpute] Iter 7: observed MAE=0.160085 rank=44
[SoftImpute] It

[SoftImpute] Max Singular Value of X_init = 101.722630
[SoftImpute] Iter 1: observed MAE=0.160184 rank=44
[SoftImpute] Iter 2: observed MAE=0.160182 rank=44
[SoftImpute] Iter 3: observed MAE=0.160154 rank=44
[SoftImpute] Iter 4: observed MAE=0.160129 rank=44
[SoftImpute] Iter 5: observed MAE=0.160111 rank=43
[SoftImpute] Iter 6: observed MAE=0.160100 rank=43
[SoftImpute] Iter 7: observed MAE=0.160094 rank=43
[SoftImpute] Iter 8: observed MAE=0.160091 rank=43
[SoftImpute] Iter 9: observed MAE=0.160088 rank=43
[SoftImpute] Iter 10: observed MAE=0.160087 rank=43
[SoftImpute] Iter 11: observed MAE=0.160087 rank=43
[SoftImpute] Iter 12: observed MAE=0.160086 rank=43
[SoftImpute] Stopped after iteration 12 for lambda=2.034453
[SoftImpute] Max Singular Value of X_init = 101.705927
[SoftImpute] Iter 1: observed MAE=0.160090 rank=44
[SoftImpute] Iter 2: observed MAE=0.160213 rank=44
[SoftImpute] Iter 3: observed MAE=0.160185 rank=44
[SoftImpute] Iter 4: observed MAE=0.160153 rank=44
[SoftImpute

[SoftImpute] Max Singular Value of X_init = 101.724793
[SoftImpute] Iter 1: observed MAE=0.160145 rank=44
[SoftImpute] Iter 2: observed MAE=0.160170 rank=44
[SoftImpute] Iter 3: observed MAE=0.160176 rank=44
[SoftImpute] Iter 4: observed MAE=0.160177 rank=44
[SoftImpute] Iter 5: observed MAE=0.160176 rank=44
[SoftImpute] Iter 6: observed MAE=0.160176 rank=44
[SoftImpute] Iter 7: observed MAE=0.160175 rank=44
[SoftImpute] Iter 8: observed MAE=0.160175 rank=44
[SoftImpute] Iter 9: observed MAE=0.160175 rank=44
[SoftImpute] Iter 10: observed MAE=0.160175 rank=44
[SoftImpute] Stopped after iteration 10 for lambda=2.034496
[SoftImpute] Max Singular Value of X_init = 101.714803
[SoftImpute] Iter 1: observed MAE=0.160078 rank=44
[SoftImpute] Iter 2: observed MAE=0.160123 rank=44
[SoftImpute] Iter 3: observed MAE=0.160141 rank=44
[SoftImpute] Iter 4: observed MAE=0.160143 rank=44
[SoftImpute] Iter 5: observed MAE=0.160143 rank=44
[SoftImpute] Iter 6: observed MAE=0.160142 rank=44
[SoftImpute] 

[SoftImpute] Max Singular Value of X_init = 101.733721
[SoftImpute] Iter 1: observed MAE=0.160207 rank=43
[SoftImpute] Iter 2: observed MAE=0.160205 rank=43
[SoftImpute] Iter 3: observed MAE=0.160197 rank=44
[SoftImpute] Iter 4: observed MAE=0.160193 rank=44
[SoftImpute] Iter 5: observed MAE=0.160191 rank=44
[SoftImpute] Iter 6: observed MAE=0.160191 rank=44
[SoftImpute] Iter 7: observed MAE=0.160190 rank=44
[SoftImpute] Iter 8: observed MAE=0.160190 rank=44
[SoftImpute] Stopped after iteration 8 for lambda=2.034674
[SoftImpute] Max Singular Value of X_init = 101.730247
[SoftImpute] Iter 1: observed MAE=0.160146 rank=44
[SoftImpute] Iter 2: observed MAE=0.160178 rank=44
[SoftImpute] Iter 3: observed MAE=0.160187 rank=44
[SoftImpute] Iter 4: observed MAE=0.160187 rank=44
[SoftImpute] Iter 5: observed MAE=0.160187 rank=44
[SoftImpute] Iter 6: observed MAE=0.160187 rank=44
[SoftImpute] Iter 7: observed MAE=0.160187 rank=44
[SoftImpute] Iter 8: observed MAE=0.160187 rank=44
[SoftImpute] St

...

c:\users\zhaon\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\zhaon\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\zhaon\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
c:\users\zhaon\appdata\local\programs\python\python36\lib\site-packages\skle

[SoftImpute] Max Singular Value of X_init = 101.727362
[SoftImpute] Iter 1: observed MAE=0.160203 rank=44
[SoftImpute] Iter 2: observed MAE=0.160194 rank=44
[SoftImpute] Iter 3: observed MAE=0.160183 rank=44
[SoftImpute] Iter 4: observed MAE=0.160174 rank=44
[SoftImpute] Iter 5: observed MAE=0.160170 rank=44
[SoftImpute] Iter 6: observed MAE=0.160169 rank=44
[SoftImpute] Iter 7: observed MAE=0.160169 rank=44
[SoftImpute] Iter 8: observed MAE=0.160168 rank=44
[SoftImpute] Iter 9: observed MAE=0.160167 rank=44
[SoftImpute] Iter 10: observed MAE=0.160167 rank=44
[SoftImpute] Iter 11: observed MAE=0.160166 rank=44
[SoftImpute] Iter 12: observed MAE=0.160166 rank=44
[SoftImpute] Iter 13: observed MAE=0.160165 rank=44
[SoftImpute] Iter 14: observed MAE=0.160165 rank=44
[SoftImpute] Iter 15: observed MAE=0.160165 rank=44
[SoftImpute] Iter 16: observed MAE=0.160165 rank=44
[SoftImpute] Iter 17: observed MAE=0.160165 rank=44
[SoftImpute] Iter 18: observed MAE=0.160165 rank=44
[SoftImpute] Iter 

[SoftImpute] Iter 31: observed MAE=0.160160 rank=44
[SoftImpute] Iter 32: observed MAE=0.160160 rank=44
[SoftImpute] Stopped after iteration 32 for lambda=2.034803
[SoftImpute] Max Singular Value of X_init = 101.705468
[SoftImpute] Iter 1: observed MAE=0.160125 rank=44
[SoftImpute] Iter 2: observed MAE=0.160140 rank=44
[SoftImpute] Iter 3: observed MAE=0.160153 rank=44
[SoftImpute] Iter 4: observed MAE=0.160158 rank=44
[SoftImpute] Iter 5: observed MAE=0.160160 rank=44
[SoftImpute] Iter 6: observed MAE=0.160160 rank=44
[SoftImpute] Iter 7: observed MAE=0.160159 rank=44
[SoftImpute] Iter 8: observed MAE=0.160158 rank=44
[SoftImpute] Iter 9: observed MAE=0.160157 rank=44
[SoftImpute] Iter 10: observed MAE=0.160156 rank=44
[SoftImpute] Iter 11: observed MAE=0.160155 rank=44
[SoftImpute] Iter 12: observed MAE=0.160154 rank=44
[SoftImpute] Iter 13: observed MAE=0.160153 rank=44
[SoftImpute] Iter 14: observed MAE=0.160153 rank=44
[SoftImpute] Iter 15: observed MAE=0.160152 rank=44
[SoftImput

[SoftImpute] Max Singular Value of X_init = 101.711069
[SoftImpute] Iter 1: observed MAE=0.160126 rank=44
[SoftImpute] Iter 2: observed MAE=0.160140 rank=44
[SoftImpute] Iter 3: observed MAE=0.160149 rank=44
[SoftImpute] Iter 4: observed MAE=0.160155 rank=44
[SoftImpute] Iter 5: observed MAE=0.160158 rank=44
[SoftImpute] Iter 6: observed MAE=0.160160 rank=44
[SoftImpute] Iter 7: observed MAE=0.160162 rank=44
[SoftImpute] Iter 8: observed MAE=0.160162 rank=44
[SoftImpute] Iter 9: observed MAE=0.160162 rank=44
[SoftImpute] Iter 10: observed MAE=0.160162 rank=44
[SoftImpute] Iter 11: observed MAE=0.160162 rank=44
[SoftImpute] Iter 12: observed MAE=0.160162 rank=44
[SoftImpute] Iter 13: observed MAE=0.160162 rank=44
[SoftImpute] Iter 14: observed MAE=0.160162 rank=44
[SoftImpute] Iter 15: observed MAE=0.160162 rank=44
[SoftImpute] Iter 16: observed MAE=0.160161 rank=44
[SoftImpute] Iter 17: observed MAE=0.160161 rank=44
[SoftImpute] Iter 18: observed MAE=0.160161 rank=44
[SoftImpute] Iter 

KeyboardInterrupt: 

In [6]:
actual_outcomes

array([ 0.88555519,  0.31054841,  0.7886831 ,  1.79606795,  0.49199063,
        1.34292969, -0.21221541,  1.98608043,  0.80390125,  2.09654268,
        1.13194444,  1.42298124,  1.36704066,  1.82685667,  1.38048471,
        2.39268856,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [7]:
individual_predictions

array([[1.18386643, 1.17060216, 1.27163192, 1.28000659, 1.11230469],
       [1.51475592, 1.69261728, 1.66620959, 1.48326595, 1.02490234],
       [0.92747467, 1.31373384, 1.30434708, 0.91010096, 1.35986328],
       [1.35697864, 1.33471192, 1.27631826, 1.4723463 , 1.06933594],
       [0.48102841, 0.72248288, 0.39635102, 0.8084688 , 0.61392212],
       [1.73675711, 1.92237859, 1.95213264, 1.4912975 , 1.37792969],
       [0.81821429, 0.48328164, 0.28166712, 1.39362352, 1.53613281],
       [1.74577741, 2.07260062, 2.09111065, 2.08507882, 1.66021729],
       [1.05741716, 0.97966286, 0.91165474, 1.17094892, 1.11816406],
       [1.77788003, 1.97531371, 1.98397232, 2.14692306, 1.72753906],
       [1.76369103, 2.2178185 , 1.82912441, 1.9211083 , 1.28515625],
       [0.8408518 , 0.72954941, 0.77772516, 1.00661304, 1.00585938],
       [1.13861943, 1.1774437 , 1.26083682, 1.58185319, 1.09326172],
       [1.2029087 , 1.21144276, 1.15179713, 1.5669569 , 1.16503906],
       [1.3329385 , 1.57263339, 1.

,MC,HR,SC,Abadie,DID,ensem1,ensem2,ensem3,ensem4
AL,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
AK,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
AZ,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
AR,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
CA,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
CO,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
CT,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
DE,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
DC,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
FL,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
